In [ ]:
import logging
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    transactions_df = spark.table("genai_demo.sas.transactions")
    customers_df = spark.table("genai_demo.sas.customers")

    # Step 1: Valid Transactions Filtering
    valid_txns_df = transactions_df.filter((F.col("Sales") > 0) & (F.col("Product") != ""))

    # Step 2: Effective Price Calculation
    trans_step2_df = valid_txns_df.withColumn("EffectivePrice", F.col("Sales") * (1 - F.col("Discount") / 100))

    # Step 3: Total Value Calculation
    trans_step3_df = trans_step2_df.withColumn("TotalValue", F.col("EffectivePrice") * F.col("Quantity"))

    # Step 4: Full Data Join
    full_data_df = trans_step3_df.join(customers_df, "CustomerID", "left").select(
        trans_step3_df["*"], customers_df["Region"], customers_df["JoinDate"]
    )

    # Step 5: Tenure Days Calculation
    trans_step5_df = full_data_df.withColumn("TenureDays", F.datediff(F.col("TransDate"), F.col("JoinDate")))

    # Step 6: Tenure Category Assignment
    trans_step6_df = trans_step5_df.withColumn(
        "TenureCategory",
        F.when(F.col("TenureDays") < 180, "New")
         .when(F.col("TenureDays") < 365, "Medium")
         .otherwise("Loyal")
    )

    # Step 7: High Value Flag
    trans_step7_df = trans_step6_df.withColumn("HighValueFlag", F.col("TotalValue") > 2000)

    # Step 8: Product Group Assignment
    trans_step8_df = trans_step7_df.withColumn(
        "ProductGroup",
        F.when(F.col("Product").isin("A", "C"), "Core").otherwise("Non-Core")
    )

    # Step 9: Z-score Standardization
    window_spec = Window.partitionBy("ProductGroup")
    standardized_df = trans_step8_df.withColumn(
        "StandardizedTotalValue",
        (F.col("TotalValue") - F.mean("TotalValue").over(window_spec)) / F.stddev("TotalValue").over(window_spec)
    )

    # Step 10: Outlier Detection
    enhanced_final_data_df = standardized_df.withColumn(
        "OutlierFlag",
        F.when(F.abs(F.col("StandardizedTotalValue")) > 2, 1).otherwise(0)
    )

    # Write the enhanced final data to Unity Catalog
    spark.sql("DROP TABLE IF EXISTS genai_demo.sas.enhanced_final_data")
    enhanced_final_data_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.sas.enhanced_final_data")

    # Generate summary statistics and reports
    # Summary Statistics by Region and Product Group
    summary_stats_df = enhanced_final_data_df.groupBy("Region", "ProductGroup").agg(
        F.mean("TotalValue").alias("MeanTotalValue"),
        F.sum("TotalValue").alias("SumTotalValue"),
        F.mean("Quantity").alias("MeanQuantity"),
        F.sum("Quantity").alias("SumQuantity"),
        F.mean("Sales").alias("MeanSales"),
        F.sum("Sales").alias("SumSales")
    )
    summary_stats_df.show()

    # Tenure Category Frequency by Region
    tenure_freq_df = enhanced_final_data_df.groupBy("TenureCategory", "Region").count()
    tenure_freq_df.show()

    # Correlation Analysis
    correlation_df = enhanced_final_data_df.select("Sales", "Discount", "Quantity", "TotalValue")
    correlation_matrix = correlation_df.corr()
    logger.info("Correlation Matrix: %s", correlation_matrix)

    # Summary by Outlier Flag
    outlier_summary_df = enhanced_final_data_df.groupBy("OutlierFlag").agg(
        F.mean("Sales").alias("MeanSales"),
        F.stddev("Sales").alias("StdDevSales"),
        F.mean("TotalValue").alias("MeanTotalValue"),
        F.stddev("TotalValue").alias("StdDevTotalValue"),
        F.mean("Quantity").alias("MeanQuantity"),
        F.stddev("Quantity").alias("StdDevQuantity")
    )
    outlier_summary_df.show()

except Exception as e:
    logger.error("An error occurred during the ETL process: %s", str(e))
